In [1]:
!pip install duckdb
import pandas as pd
import duckdb

customers = pd.read_csv('customers.csv')
products = pd.read_csv('products.csv')
stores = pd.read_csv('store.csv')
transactions = pd.read_csv('transactions.csv')

# Quick look at each
print(customers.shape, products.shape, stores.shape, transactions.shape)

(200, 7) (50, 6) (5, 4) (5000, 8)


In [2]:
customers.head()

,CustomerID,FirstName,LastName,Gender,BirthDate,City,JoinDate
0,C001,Michael,Davis,M,1996-09-11,Osborneport,2022-09-25
1,C002,Michael,Miller,M,1959-08-18,New Gabrielleport,2020-11-03
2,C003,Carol,Hays,F,2005-04-19,Port Allen,2024-02-12
3,C004,Joseph,Ward,M,1992-06-16,East Edgarborough,2024-09-09
4,C005,Jamie,Salinas,M,1992-06-18,Port Kimberly,2022-02-24


In [3]:
products.head()

,ProductID,ProductName,Category,SubCategory,UnitPrice,CostPrice
0,P001,Like Camera,Electronics,Camera,1673.69,1323.38
1,P002,Audience Television,Electronics,Television,818.76,527.62
2,P003,Here Footwear,Fashion,Footwear,337.63,169.03
3,P004,Four Accessories,Fashion,Accessories,1853.77,1363.52
4,P005,Knowledge Bags,Fashion,Bags,1396.22,1004.09


In [4]:
stores.head()

,StoreID,StoreName,City,Region
0,S001,MegaMart Jimenezborough,Jimenezborough,South
1,S002,MegaMart Peckmouth,Peckmouth,East
2,S003,MegaMart New Michele,New Michele,West
3,S004,MegaMart Brianahaven,Brianahaven,North
4,S005,MegaMart Johnmouth,Johnmouth,East


In [5]:
transactions.head()

,TransactionID,Date,CustomerID,ProductID,StoreID,Quantity,Discount,PaymentMethod
0,T00001,2024-06-18,C160,P014,S003,1,0.10,Bank Transfer
1,T00002,2023-11-02,C171,P030,S004,3,0.15,Bank Transfer
2,T00003,2024-03-28,C142,P002,S002,2,0.15,Mobile Money
3,T00004,2024-06-15,C174,P050,S002,5,0.10,Mobile Money
4,T00005,2024-08-29,C141,P036,S001,3,0.10,Credit Card


In [6]:
# Connecting to DuckDB in-memory
con = duckdb.connect()

In [7]:
# Registering DataFrames as DuckDB tables
con.register('customers', customers)
con.register('products', products)
con.register('stores', stores)
con.register('transactions', transactions)

In [8]:
# check
print(con.execute("DESCRIBE transactions").df())
print(con.execute("DESCRIBE customers").df())

     column_name column_type null   key default extra
0  TransactionID     VARCHAR  YES  None    None  None
1           Date     VARCHAR  YES  None    None  None
2     CustomerID     VARCHAR  YES  None    None  None
3      ProductID     VARCHAR  YES  None    None  None
4        StoreID     VARCHAR  YES  None    None  None
5       Quantity      BIGINT  YES  None    None  None
6       Discount      DOUBLE  YES  None    None  None
7  PaymentMethod     VARCHAR  YES  None    None  None
  column_name column_type null   key default extra
0  CustomerID     VARCHAR  YES  None    None  None
1   FirstName     VARCHAR  YES  None    None  None
2    LastName     VARCHAR  YES  None    None  None
3      Gender     VARCHAR  YES  None    None  None
4   BirthDate     VARCHAR  YES  None    None  None
5        City     VARCHAR  YES  None    None  None
6    JoinDate     VARCHAR  YES  None    None  None


In [9]:
# Customers who joined in 2024

duckdb.query("""
SELECT *
FROM customers
WHERE EXTRACT(YEAR FROM CAST(JoinDate AS DATE)) = 2024
LIMIT 5
""").df()

,CustomerID,FirstName,LastName,Gender,BirthDate,City,JoinDate
0,C003,Carol,Hays,F,2005-04-19,Port Allen,2024-02-12
1,C004,Joseph,Ward,M,1992-06-16,East Edgarborough,2024-09-09
2,C017,Vanessa,Anderson,F,1982-03-11,Peterside,2024-09-25
3,C018,Breanna,Davis,M,1994-08-19,East William,2024-11-12
4,C020,Ryan,Cole,F,1982-06-26,Travisstad,2024-04-19


In [10]:
# Total Number of Male and Female Customers

duckdb.query("""
select
case
when gender = 'M' then 'MALE'
when gender = 'F' then 'FEMALE'
end as Gender,
count(*) as Count
from customers
group by 1
order by Count DESC
""").df()

,Gender,Count
0,MALE,113
1,FEMALE,87


In [11]:
# Customers who have spent more than $10,000 in total

duckdb.query("""
SELECT 
    c.CustomerID,
    c.FirstName || ' ' || c.LastName AS FullName,
    SUM((p.UnitPrice * t.Quantity) - t.Discount) AS TotalSpent
FROM customers c
JOIN transactions t ON c.CustomerID = t.CustomerID
JOIN products p ON t.ProductID = p.ProductID
GROUP BY c.CustomerID, FullName
HAVING TotalSpent > 10000
ORDER BY TotalSpent DESC
""").df()

,CustomerID,FullName,TotalSpent
0,C012,Dale Perry,127918.10
1,C110,Travis Peters,122452.34
2,C085,Juan Ramirez,121536.97
3,C186,Vicki Guzman,110839.19
4,C168,Richard Jones,110602.99
...,...,...,...
195,C160,Meagan Macdonald,47580.23
196,C114,Miranda Rodriguez,45023.19
197,C011,Haley Williams,43879.12
198,C015,Matthew Molina,41851.15


In [12]:
# Find customers who joined in the last 90 days and already made at least 1 purchase

duckdb.query("""
Select c.FirstName || ' ' || c.LastName AS FullName,
c.JoinDate, count(TransactionID) as "Total Transactions"
from customers c
join transactions t
on c.CustomerID = t.CustomerID
where CAST(c.JoinDate as DATE) > CURRENT_DATE() - INTERVAL 90 DAY
group by FullName,JoinDate
""").df()

,FullName,JoinDate,Total Transactions
0,Scott Howell,2025-08-28,21
1,Kimberly Stokes,2025-08-07,22
2,Amber Graham,2025-08-20,26
3,Ashley Gibson,2025-08-26,23
4,Jamie Webb,2025-07-31,32
5,Catherine Rice,2025-07-23,23
6,Dennis Wallace,2025-08-19,26
7,Christina Dominguez,2025-07-28,31
8,Lisa Ruiz,2025-08-22,16
9,Erin Rogers,2025-08-20,30


In [13]:
# Customers haven’t made any transactions in the last 3 months

duckdb.query("""
Select c.FirstName || ' ' || c.LastName AS FullName,
c.JoinDate, count(t.transactionID) AS NumofTransactions
from customers c
left join transactions t
on c.CustomerID = t.CustomerID
where CAST(c.JoinDate as DATE) < CURRENT_DATE() - INTERVAL 180 DAY
group by FullName,c.JoinDate
having count(t.transactionID) is NULL
order by c.JoinDate DESC
""").df()

,FullName,JoinDate,NumofTransactions


In [14]:
# City having the most customers

duckdb.query("""
select count(customerID) as totalcustomers, City
from customers
group by city
order by totalcustomers desc
""").df()

,totalcustomers,City
0,1,Leburgh
1,1,Juliefurt
2,1,Bradyshire
3,1,Shepherdburgh
4,1,New David
...,...,...
195,1,Lake Lindsay
196,1,Wilsonland
197,1,Lake Melissaborough
198,1,South Marcusberg


In [15]:
# Customers visits by city

duckdb.query("""
select s.city, count(c.customerID) as customersvisit
from stores s
join transactions as t
on t.storeId = s.storeid
join customers as c
on c.customerID = t.customerID
group by s.city
order by customersvisit desc
""").df()

,City,customersvisit
0,Jimenezborough,1014
1,New Michele,1013
2,Brianahaven,1011
3,Peckmouth,1009
4,Johnmouth,953


In [16]:
# Customer Demographics : Average age of customers buying each category

duckdb.query("""
select p.category,
avg(DATE_DIFF('year',CAST(c.BirthDate AS DATE),CURRENT_DATE)) as AverageAge
from transactions t
join products p on t.productid = p.productid
join customers c on t.customerid = c.customerid
group by p.category
order by AverageAge DESC
""").df()

,Category,AverageAge
0,Groceries,45.220624
1,Fashion,45.161099
2,Electronics,43.988895


In [17]:
# All products under Electronics Category

duckdb.query("""
SELECT *
FROM products
WHERE Category = 'Electronics';
""").df()

,ProductID,ProductName,Category,SubCategory,UnitPrice,CostPrice
0,P001,Like Camera,Electronics,Camera,1673.69,1323.38
1,P002,Audience Television,Electronics,Television,818.76,527.62
2,P007,Understand Camera,Electronics,Camera,1474.12,764.02
3,P010,Step Smartphone,Electronics,Smartphone,1194.12,838.95
4,P011,Either Headphones,Electronics,Headphones,248.57,190.67
5,P012,Above Camera,Electronics,Camera,1193.09,818.24
6,P017,Find Headphones,Electronics,Headphones,1246.11,833.52
7,P018,Democratic Smartphone,Electronics,Smartphone,1079.89,703.83
8,P019,Traditional Laptop,Electronics,Laptop,1762.92,1346.48
9,P024,Him Smartphone,Electronics,Smartphone,1487.41,813.07


In [18]:
# Top 3 best-selling products (by quantity)

duckdb.query("""
SELECT 
    p.ProductID,
    p.ProductName,
    SUM(t.Quantity) AS TotalQuantity
FROM products p
JOIN transactions t ON p.ProductID = t.ProductID
GROUP BY p.ProductID, p.ProductName
ORDER BY TotalQuantity DESC
LIMIT 3;
""").df()

,ProductID,ProductName,TotalQuantity
0,P029,Road Clothing,346.0
1,P022,How Vegetables,343.0
2,P002,Audience Television,341.0


In [19]:
# Average quantity purchased per transaction for each product

duckdb.query("""
SELECT 
    p.ProductID,
    p.ProductName,
    AVG(t.Quantity) AS AvgQuantityPerTransaction
FROM products p
JOIN transactions t ON p.ProductID = t.ProductID
GROUP BY p.ProductID, p.ProductName
LIMIT 10;
""").df()

,ProductID,ProductName,AvgQuantityPerTransaction
0,P002,Audience Television,3.247619
1,P036,New Watches,2.927083
2,P035,Chair Laptop,3.182796
3,P045,Set Dairy,2.972973
4,P018,Democratic Smartphone,2.821053
5,P048,Nor Bags,3.192308
6,P034,Everything Laptop,3.158879
7,P016,House Accessories,2.791209
8,P007,Understand Camera,2.951807
9,P039,Church Clothing,2.893204


In [20]:
# Profit per product

duckdb.query("""
SELECT 
    p.ProductID,
    p.ProductName,
    SUM((p.UnitPrice - p.CostPrice) * t.Quantity - t.Discount) AS TotalProfit
FROM products p
JOIN transactions t ON p.ProductID = t.ProductID
GROUP BY p.ProductID, p.ProductName
LIMIT 10;
""").df()

,ProductID,ProductName,TotalProfit
0,P002,Audience Television,99270.69
1,P036,New Watches,93770.03
2,P035,Chair Laptop,108566.25
3,P045,Set Dairy,288582.85
4,P018,Democratic Smartphone,100776.48
5,P048,Nor Bags,139950.07
6,P034,Everything Laptop,61501.64
7,P016,House Accessories,103520.76
8,P007,Understand Camera,173968.30
9,P039,Church Clothing,42513.07


In [21]:
# Total Sales Revenue

duckdb.query("""
SELECT 
    SUM(p.UnitPrice * t.Quantity - t.Discount) AS TotalRevenue
FROM transactions t
JOIN products p ON t.ProductID = p.ProductID;
""").df()

,TotalRevenue
0,15474792.32


In [22]:
# Sales by category

duckdb.query("""
SELECT 
    p.Category,
    SUM(p.UnitPrice * t.Quantity - t.Discount) AS Revenue
FROM products p
JOIN transactions t ON p.ProductID = t.ProductID
GROUP BY p.Category;
""").df()

,Category,Revenue
0,Groceries,1897960.02
1,Fashion,6740203.62
2,Electronics,6836628.68


In [23]:
# Top 5 customers by spend

duckdb.query("""
SELECT 
    c.CustomerID,
    c.FirstName || ' ' || c.LastName AS FullName,
    SUM(p.UnitPrice * t.Quantity - t.Discount) AS TotalSpent
FROM customers c
JOIN transactions t ON c.CustomerID = t.CustomerID
JOIN products p ON t.ProductID = p.ProductID
GROUP BY c.CustomerID, FullName
ORDER BY TotalSpent DESC
LIMIT 5;
""").df()

,CustomerID,FullName,TotalSpent
0,C012,Dale Perry,127918.10
1,C110,Travis Peters,122452.34
2,C085,Juan Ramirez,121536.97
3,C186,Vicki Guzman,110839.19
4,C168,Richard Jones,110602.99


In [24]:
# Monthly Sales trend

duckdb.query("""
SELECT 
    STRFTIME(CAST(t.Date AS DATE), '%Y-%m') AS Month,
    SUM(p.UnitPrice * t.Quantity - t.Discount) AS MonthlyRevenue
FROM transactions t
JOIN products p ON t.ProductID = p.ProductID
GROUP BY Month
ORDER BY Month
LIMIT 5;
""").df()

,Month,MonthlyRevenue
0,2023-09,411948.20
1,2023-10,605199.01
2,2023-11,637511.18
3,2023-12,705268.78
4,2024-01,630533.43


In [25]:
# Payment Method Sales trend

duckdb.query("""
SELECT 
    t.PaymentMethod,
    SUM(p.UnitPrice * t.Quantity - t.Discount) AS Revenue
FROM transactions t
JOIN products p ON t.ProductID = p.ProductID
GROUP BY t.PaymentMethod;
""").df()

,PaymentMethod,Revenue
0,Bank Transfer,3683980.01
1,Mobile Money,3846521.34
2,Credit Card,3973995.26
3,Cash,3970295.71


In [26]:
# Store-wise performance

duckdb.query("""
SELECT 
    s.StoreName,
    SUM(p.UnitPrice * t.Quantity - t.Discount) AS Revenue
FROM stores s
JOIN transactions t ON s.StoreID = t.StoreID
JOIN products p ON t.ProductID = p.ProductID
GROUP BY s.StoreName;
""").df()

,StoreName,Revenue
0,MegaMart Brianahaven,3143179.88
1,MegaMart New Michele,3156011.62
2,MegaMart Peckmouth,2980119.26
3,MegaMart Jimenezborough,3121699.28
4,MegaMart Johnmouth,3073782.28


In [27]:
# Region-wise performance

duckdb.query("""
SELECT 
    s.Region,
    SUM(p.UnitPrice * t.Quantity - t.Discount) AS Revenue
FROM stores s
JOIN transactions t ON s.StoreID = t.StoreID
JOIN products p ON t.ProductID = p.ProductID
GROUP BY s.Region;
""").df()

,Region,Revenue
0,North,3143179.88
1,East,6053901.54
2,West,3156011.62
3,South,3121699.28


In [29]:
# Sales growth percentage

duckdb.query("""
WITH monthly_sales AS (
    SELECT 
        STRFTIME(CAST(t.Date AS DATE), '%Y-%m') AS Month,
        SUM(p.UnitPrice * t.Quantity - t.Discount) AS Revenue
    FROM transactions t
    JOIN products p ON t.ProductID = p.ProductID
    GROUP BY Month
)
SELECT
    Month,
    Revenue,
    (Revenue - LAG(Revenue, 1) OVER (ORDER BY Month)) / NULLIF(LAG(Revenue, 1) OVER (ORDER BY Month), 0) * 100 AS GrowthPercentage
FROM monthly_sales;
""").df()

,Month,Revenue,GrowthPercentage
0,2023-09,411948.20,NaN
1,2023-10,605199.01,46.911434
2,2023-11,637511.18,5.339098
3,2023-12,705268.78,10.628457
4,2024-01,630533.43,-10.596719
5,2024-02,583625.30,-7.439436
6,2024-03,599524.34,2.724186
7,2024-04,640978.17,6.914453
8,2024-05,613915.68,-4.222061
9,2024-06,682244.85,11.130058


In [31]:
# Year-over-year sales

duckdb.query("""
SELECT 
    STRFTIME(CAST(t.Date AS DATE), '%Y') AS Year,
    SUM(p.UnitPrice * t.Quantity - t.Discount) AS Revenue
FROM transactions t
JOIN products p ON t.ProductID = p.ProductID
GROUP BY Year
ORDER BY Year;
""").df()

,Year,Revenue
0,2023,2359927.17
1,2024,7597950.01
2,2025,5516915.14


In [32]:
#  The most popular payment method per region

duckdb.query("""
SELECT 
    s.Region,
    t.PaymentMethod,
    COUNT(*) AS CountMethod
FROM transactions t
JOIN stores s ON t.StoreID = s.StoreID
GROUP BY s.Region, t.PaymentMethod
QUALIFY ROW_NUMBER() OVER (PARTITION BY s.Region ORDER BY CountMethod DESC) = 1;
""").df()

,Region,PaymentMethod,CountMethod
0,East,Cash,521
1,North,Mobile Money,267
2,South,Mobile Money,270
3,West,Cash,260
